In [1]:
import pandas as pd

In [15]:
customers = pd.read_csv("customers.csv")
products = pd.read_csv("products.csv")
sellers = pd.read_csv("sellers.csv")
sales = pd.read_csv("sales_fact.csv")

In [25]:
def clean_columns(df):
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_")
        .str.replace("-", "_")
    )
    return df

customers = clean_columns(customers)
products = clean_columns(products)
sellers = clean_columns(sellers)
sales = clean_columns(sales)

In [27]:
if "order_date" in sales.columns:
    sales["order_date"] = pd.to_datetime(sales["order_date"], errors="coerce")

# Ensure numeric columns are numeric
for col in ["total_amount", "discount_amount", "shipping_cost", "rating"]:
    if col in sales.columns:
        sales[col] = pd.to_numeric(sales[col], errors="coerce")

In [29]:
if "order_date" in sales.columns:
    sales["order_year"] = sales["order_date"].dt.year
    sales["order_month"] = sales["order_date"].dt.month

if {"total_amount", "discount_amount", "shipping_cost"}.issubset(sales.columns):
    sales["net_amount"] = (
        sales["total_amount"]
        - sales["discount_amount"].fillna(0)
        - sales["shipping_cost"].fillna(0)
    )


In [31]:
for df in [customers, products, sellers, sales]:
    for col in df.select_dtypes(include="object").columns:
        df[col] = df[col].fillna("Unknown")

# Replace missing numeric values with 0
for df in [customers, products, sellers, sales]:
    for col in df.select_dtypes(include="number").columns:
        df[col] = df[col].fillna(0)

In [34]:
if "total_amount" in sales.columns:
    print("Negative total_amount rows:\n", sales[sales["total_amount"] < 0])

if "rating" in sales.columns:
    print("Invalid ratings:\n", sales[(sales["rating"] < 1) | (sales["rating"] > 5)])


Negative total_amount rows:
         order_id customer_id product_id seller_id  quantity order_date  \
89        ORD_90   CUST_4078  PROD_7565  SELL_729         5 2023-01-01   
102      ORD_103   CUST_3686   PROD_205  SELL_820         3 2023-01-01   
144      ORD_145   CUST_4255  PROD_1316  SELL_425         1 2023-01-02   
266      ORD_267    CUST_524  PROD_6467   SELL_56         3 2023-01-02   
291      ORD_292   CUST_3561  PROD_6321  SELL_563         1 2023-01-03   
...          ...         ...        ...       ...       ...        ...   
49645  ORD_49646    CUST_932  PROD_1765   SELL_28         1 2023-12-28   
49741  ORD_49742    CUST_765  PROD_8750  SELL_273         1 2023-12-29   
49760  ORD_49761   CUST_4448  PROD_6124  SELL_640         2 2023-12-29   
49895  ORD_49896   CUST_4475  PROD_9481  SELL_928         2 2023-12-30   
49920  ORD_49921   CUST_4473  PROD_5523  SELL_127         5 2023-12-30   

       shipping_cost  discount_amount    payment_method  total_amount  \
89       

In [36]:
print("Unique customer IDs:", customers["customer_id"].nunique(), "/", len(customers))
print("Unique product IDs:", products["product_id"].nunique(), "/", len(products))
print("Unique seller IDs:", sellers["seller_id"].nunique(), "/", len(sellers))

Unique customer IDs: 5000 / 5000
Unique product IDs: 10000 / 10000
Unique seller IDs: 1000 / 1000


In [38]:
missing_customers = set(sales["customer_id"]) - set(customers["customer_id"])
missing_products = set(sales["product_id"]) - set(products["product_id"])
missing_sellers = set(sales["seller_id"]) - set(sellers["seller_id"])

print("Missing customers in sales_fact:", len(missing_customers))
print("Missing products in sales_fact:", len(missing_products))
print("Missing sellers in sales_fact:", len(missing_sellers))

Missing customers in sales_fact: 0
Missing products in sales_fact: 0
Missing sellers in sales_fact: 0


In [40]:
customers.to_csv("customers_clean.csv", index=False)
products.to_csv("products_clean.csv", index=False)
sellers.to_csv("sellers_clean.csv", index=False)
sales.to_csv("sales_fact_clean.csv", index=False)

print("\n✅ Cleaning complete! Cleaned files saved as *_clean.csv")


✅ Cleaning complete! Cleaned files saved as *_clean.csv
